# TASK1 赛题理解

1 赛题以预测二手车的交易价格为任务，数据集报名后可见并可下载，该数据来自某交易平台的二手车交易记录，总数据量超过40w，包含31列变量信息，其中15列为匿名变量。为了保证比赛的公平性，将会从中抽取15万条作为训练集，5万条作为测试集A，5万条作为测试集B，同时会对name、model、brand和regionCode等信息进行脱敏。

训练集各特征train.csv

name - 汽车编码
regDate - 汽车注册时间
model - 车型编码
brand - 品牌
bodyType - 车身类型
fuelType - 燃油类型
gearbox - 变速箱
power - 汽车功率
kilometer - 汽车行驶公里
notRepairedDamage - 汽车有尚未修复的损坏
regionCode - 看车地区编码
seller - 销售方
offerType - 报价类型
creatDate - 广告发布时间
price - 汽车价格
v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14'

本赛题的评价标准为MAE(Mean Absolute Error):

2 赛题分析

此题为传统的数据挖掘问题，通过数据科学以及机器学习深度学习的办法来进行建模得到结果。/
此题是一个典型的回归问题。/
主要应用xgb、lgb、catboost，以及pandas、numpy、matplotlib、seabon、sklearn、keras等等数据挖掘常用库或者框架来进行数据挖掘任务。/
通过EDA来挖掘数据的联系和自我熟悉数据。/

数据读取

In [1]:
import pandas as pd
import numpy as np

In [2]:
Train_data = pd.read_csv('F:\二手车\car_train_0110.csv', sep=' ')
Test_data = pd.read_csv('F:\二手车\car_testA_0110.csv', sep=' ')
print('Train data shape:',Train_data.shape)
print('TestA data shape:',Test_data.shape)

Train data shape: (250000, 40)
TestA data shape: (50000, 39)


In [3]:
 Train_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_14,v_15,v_16,v_17,v_18,v_19,v_20,v_21,v_22,v_23
0,134890,734,20160002,13.0,9,NaN,0.0,1.0,0,15.0,...,0.092139,0.000000,18.763832,-1.512063,-1.008718,-12.100623,-0.947052,9.077297,0.581214,3.945923
1,306648,196973,20080307,72.0,9,7.0,5.0,1.0,173,15.0,...,0.001070,0.122335,-5.685612,-0.489963,-2.223693,-0.226865,-0.658246,-3.949621,4.593618,-1.145653
2,340675,25347,20020312,18.0,12,3.0,0.0,1.0,50,12.5,...,0.064410,0.003345,-3.295700,1.816499,3.554439,-0.683675,0.971495,2.625318,-0.851922,-1.246135
3,57332,5382,20000611,38.0,8,7.0,0.0,1.0,54,15.0,...,0.069231,0.000000,-3.405521,1.497826,4.782636,0.039101,1.227646,3.040629,-0.801854,-1.251894
4,265235,173174,20030109,87.0,0,5.0,5.0,1.0,131,3.0,...,0.000099,0.001655,-4.475429,0.124138,1.364567,-0.319848,-1.131568,-3.303424,-1.998466,-1.279368


In [4]:
Test_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_14,v_15,v_16,v_17,v_18,v_19,v_20,v_21,v_22,v_23
0,720326,505,20060505,19.0,13,7.0,0.0,1.0,90,8.0,...,0.083340,0.105382,-5.998993,0.147048,-1.902847,0.348990,2.324961,3.343910,4.048742,-1.431822
1,714316,1836,20010301,5.0,5,3.0,4.0,1.0,75,15.0,...,0.074478,0.000000,-3.287221,2.081317,2.937052,-0.123018,1.202395,3.570743,-1.180587,-1.348598
2,704693,212291,20170610,6.0,18,NaN,5.0,0.0,150,15.0,...,0.002032,0.000000,4.368218,8.252188,-4.136109,-13.334970,-4.444620,-0.706978,-1.720218,3.569112
3,624972,1345,19820005,215.0,32,7.0,0.0,1.0,0,6.0,...,0.098806,0.100883,-2.537486,0.513955,4.414962,0.357685,2.700732,5.323602,6.085956,-0.900585
4,669753,1428,20060205,30.0,4,7.0,5.0,1.0,122,15.0,...,0.088397,0.002509,-6.197633,-0.191814,-1.224360,-0.326985,2.254931,4.183037,-2.574004,0.014203


总结：通过查看训练数据和测试数据可以看出，saleID和name应该是无关变量，表征交易ID和交易名称，在后续特征处理时可以删去，regDate,model,brand,bodytype,fueltype,gearbox, notRepairedDamage,regionCode,seller,offerType,creatDate是离散型特征，其他为连续性特征，离散型特征可以离散化，连续性特征可以归一化。并且数据中存在缺失值，可以用-1填充或者中位数填充。汽车售卖时间和注册时间之差可以作为使用时间，由于时间特征不规范，所以考虑不用天数而是用年数作为新的特征。